In [1]:
from bs4 import BeautifulSoup
import requests
from requests import get
import urllib.parse
from urllib.error import HTTPError
import json
import time
import pandas as pd
import numpy as np

Scraping bestplaces.net's political leanings which is based on recent voting in 
national elections, federal campaign contributions by local residents, and 
consumer personality profiles.

In [2]:
top_cities = pd.read_csv('Top5000Population.csv')
states = pd.read_csv('data.csv')
states['Cities'] = np.empty((len(states), 0)).tolist()
top_cities['State']=np.empty((len(top_cities)))

In [3]:
for state_index, state_row in states.iterrows():
    for city_index, city_row in top_cities.iterrows():
        if city_row['Code'] == state_row['Code']:
            state_row['Cities'].append(city_row['City'])

In [4]:
def collect_affiliation_data(city,state):
    url = 'https://www.bestplaces.net/voting/city/'+state+'/'+city
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    container = html_soup.find_all('div', class_ = 'col-md-12')
    output = container[1].find('b').text
    bad_chars = ['.', ','] 
    for i in bad_chars : 
        output = output.replace(i, '') 
    try: 
        return(output.split(' is ')[1].lower())
    except:
        return('NA')

In [5]:
outputlist = []
citylist = []
statelist = []

In [ ]:

for state_index, state_row in states.iterrows():
    state = state_row['State']
    for city in state_row['Cities']:
        cityname = city.replace(" ","_")
        try:
            outputlist.append(collect_affiliation_data(cityname,state))
        except:
            outputlist.append('NA')
        citylist.append(city)
        statelist.append(state_row['State'])
        print(state)
        print(city)


Alabama
Birmingham 
Alabama
Montgomery 
Alabama
Mobile 
Alabama
Huntsville 


In [29]:
len(outputlist)

186

In [30]:
outputdf = pd.DataFrame(outputlist)
citydf = pd.DataFrame(citylist)
statedf = pd.DataFrame(statelist)

In [31]:
df_out = pd.concat([outputdf,citydf,statedf],axis=1)

In [33]:
df_out.head()

,0,0,0
0,moderately liberal,New York,New York
1,moderately liberal,Buffalo,New York
2,moderately liberal,Rochester,New York
3,moderately liberal,Yonkers,New York
4,somewhat liberal,Syracuse,New York


In [34]:

df_out.to_csv('output.csv')